# Importing ORACC Data from corpus.json
by Niek Veldhuis

February 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely.
* look for `'$ n lines broken'` lines. 

'$' line has 'node' = 'd' with 'type' = 'nonx' and 'strict' = '1'. There are three content nodes: 'extent', 'scope' and 'state'.

# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) JSON files that contain textual data and produce a `.csv` file with the relevant data for use in computational text analysis. This comes in the place of scraping the published `html`. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file).

The resulting data file may include various elements of the ORACC data structure. The current code will output a file with the following fields: 
* textid
* line number
* lemmatization
* bases

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. This notebook is written for exploratory purposes, using a list of 106 P, Q, X numbers (in `ob_lists_wood.txt`). Downloading the `.json` files takes between 30 and 45 seconds. The rest of the script is reasonably fast. With larger lists of text IDs the script will obviously take longer. 

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

In [1]:
import pandas as pd
import ijson
import urllib.request
import re
import tqdm

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

In [2]:
filename = input('Filename: ')

Filename: ob_lists_wood.txt


In [3]:
textids = 'text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
prefix = [x[:-7] + 'corpusjson/' + x[-7:] for x in pqxnos]
pqxnos = [x[-7:] for x in pqxnos]
pqxnos[:5], prefix[:5]


(['Q000039', 'P117395', 'P117404', 'P128345', 'P224980'],
 ['dcclt/corpusjson/Q000039',
  'dcclt/corpusjson/P117395',
  'dcclt/corpusjson/P117404',
  'dcclt/corpusjson/P128345',
  'dcclt/corpusjson/P224980'])

# Parse
The function `oraccjasonparser()` takes one argument (the **url** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only included lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [4]:
def oraccjasonparser(url):
    d = urllib.request.urlopen(url)
    parser = ijson.parse(d)
    word_l = []
    word_d = {}
    line_start = False
    word_start = False
    nonx = False
    for prefix, event, value in parser:
        if prefix == 'textid':
            id_text = value
#            print("parsing " + value)
        if prefix.endswith('.type'):
            if value == 'line-start':
                line_start = True
            else:
                line_start = False
        if line_start:
            if prefix.endswith('.ref') and not word_start:
                id_line = value
            if prefix.endswith('.label'):
                label = value
        if prefix.endswith('node'):
            if value == 'l':
                word_start = True
                if not word_d == {}:
                    word_l.append(word_d)
                word_d = {}
                word_d['id_text'] = id_text
                word_d['id_line'] = id_line
                word_d['label'] = label
            else:
                word_start = False
        if word_start:
            if prefix.endswith('.ref'):
                word_d['id_word'] = value
            if prefix.endswith('.sig'):
                word_d['signature'] = value
            if '.f.' in prefix:
                category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                word_d[category] = value
        if prefix.endswith('.type'):
            if value == 'nonx':
                nonx = True
            else:
                nonx = False
        if nonx:
            if prefix.endswith('.ref'):
                id_line = value
            if prefix.endswith('.strict'):
                if value == '1':           # select only 'strict' $ lines
                    if not word_d == {}:
                        word_l.append(word_d)
                    word_d = {}
                    word_d['id_line'] = id_line
                    word_d['id_text'] = id_text
                else:
                    nonx = False
            if prefix.endswith('.extent'):
                word_d['extent'] = value
            if prefix.endswith('.scope'):
                word_d['scope'] = value
            if prefix.endswith('.state'):
                word_d['state'] = value

    word_l.append(word_d)
    return(word_l)

# Call the Parser Function for Each Textid

In [5]:
url_prefix = "http://oracc.museum.upenn.edu/"
word_l = []
for id_text in tqdm.tqdm(prefix):
    url = url_prefix + id_text + '.json'
    word_l.extend(oraccjasonparser(url))

100%|██████████| 106/106 [00:46<00:00,  2.96it/s]


# Transform the Data into a DataFrame

In [6]:
words = pd.DataFrame(word_l)
words.head()

,base,cf,cont,epos,extent,form,gw,id_line,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,signature,state
0,{ŋeš}taškarin,taškarin,NaN,N,NaN,{ŋeš}taškarin,boxwood,Q000039.1,Q000039,Q000039.1.1,1,sux,~,NaN,taškarin,N,NaN,"box tree, boxwood",@dcclt%sux:{ŋeš}taškarin=taškarin[boxwood//box...,NaN
1,{ŋeš}esi,esi,NaN,N,NaN,{ŋeš}esi,tree,Q000039.2,Q000039,Q000039.2.1,2,sux,~,NaN,esi,N,NaN,ebony,@dcclt%sux:{ŋeš}esi=esi[tree//ebony]N'N$esi/{ŋ...,NaN
2,ŋeš-nu₁₁,ŋešnu,NaN,N,NaN,{ŋeš}nu₁₁,tree,Q000039.3,Q000039,Q000039.3.1,3,sux,~,NaN,ŋešnu,N,NaN,a tree,@dcclt%sux:{ŋeš}nu₁₁=ŋešnu[tree//a tree]N'N$ŋe...,NaN
3,{ŋeš}ha-lu-ub₂,halub,NaN,N,NaN,{ŋeš}ha-lu-ub₂,tree,Q000039.4,Q000039,Q000039.4.1,4,sux,~,NaN,halub,N,NaN,tree,@dcclt%sux:{ŋeš}ha-lu-ub₂=halub[tree//tree]N'N...,NaN
4,{ŋeš}šag₄-kal,šagkal,NaN,N,NaN,{ŋeš}šag₄-kal,tree,Q000039.5,Q000039,Q000039.5.1,5,sux,~,NaN,šagkal,N,NaN,tree,@dcclt%sux:{ŋeš}šag₄-kal=šagkal[tree//tree]N'N...,NaN


# Remove Spaces and Commas from Guide Word and Sence
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [7]:
#words['extent'] = words['extent'].fillna('0')
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]
#words['extent'] = ['0' if x == '' else x for x in words['extent']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

In [8]:
words[words['label'] == '']

,base,cf,cont,epos,extent,form,gw,id_line,id_text,id_word,label,lang,morph,norm,norm0,pos,scope,sense,signature,state
1309,,,,,3,,,Q000039.732,Q000039,,,,,,,,line,,,missing
1320,,,,,1,,,Q000039.743,Q000039,,,,,,,,line,,,missing
1422,,,,,1,,,P225065.4,P225065,,,,,,,,line,,,missing
1436,,,,,2,,,P225126.4,P225126,,,,,,,,line,,,missing
1555,,,,,10,,,P235262.4,P235262,,,,,,,,line,,,missing
1583,,,,,rest,,,P235262.21,P235262,,,,,,,,column,,,missing
1584,,,,,n,,,P235262.23,P235262,,,,,,,,line,,,missing
1585,,,,,n,,,P235262.25,P235262,,,,,,,,line,,,missing
1586,,,,,n,,,P235262.28,P235262,,,,,,,,line,,,missing
1587,,,,,8,,,P235262.30,P235262,,,,,,,,line,,,missing


# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [9]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [10]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + 
                  words['gw'][i] + ']' +
                  words['pos'][i] if not words['lemma'][i] == ''
                  else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' or words['label'][i] == '' else words['form'][i] 
                  for i in range(len(words))]
#words['label'] = [label if not label == '' else 'none' for label in words['label']]
lemmas = words[['lemma', 'base', 'id_text', 'id_line', 'id_word', 'label', 'extent', 'scope']]
lemmas.head()

,lemma,base,id_text,id_line,id_word,label,extent,scope
0,taškarin[boxwood]N,{ŋeš}taškarin,Q000039,Q000039.1,Q000039.1.1,1,,
1,esi[tree]N,{ŋeš}esi,Q000039,Q000039.2,Q000039.2.1,2,,
2,ŋešnu[tree]N,ŋeš-nu₁₁,Q000039,Q000039.3,Q000039.3.1,3,,
3,halub[tree]N,{ŋeš}ha-lu-ub₂,Q000039,Q000039.4,Q000039.4.1,4,,
4,šagkal[tree]N,{ŋeš}šag₄-kal,Q000039,Q000039.5,Q000039.5.1,5,,


## Group by Line

In [11]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join}).reset_index()

In [12]:
lines = lines[['id_line', 'label', 'lemma', 'base', 'extent', 'scope']]
lines

,id_line,label,lemma,base,extent,scope
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,,
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,,
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,,
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,,
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,,
5,P117404.4,o 3,ig[door]N i[oil]N,{ŋeš}ig i₃,,
6,P128345.2,o 1,garig[comb]N siki[hair]N,{ŋeš}ga-rig₂ siki,,
7,P128345.3,o 2,garig[comb]N siki-siki[NA]NA,{ŋeš}ga-rig₂ siki-siki,,
8,P128345.4,o 3,garig[comb]N saŋdu[head]N,{ŋeš}ga-rig₂ saŋ-du,,
9,P224980.4,o i 1,gigir[chariot]N,{ŋeš}gigir,,


## Save in CSV Format

In [13]:
filename = filename[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8')